In [92]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
import datetime
import time
import math
import warnings
warnings.filterwarnings("ignore")
import glob

In [93]:
def print_heads_tails(df, h=True,t=True):
    print(f'Data has shape: {df.shape}')
    if h:
        display(df.head(2))

    print('...')

    if t:
        display(df.tail(2))

In [94]:
def read_labels():
    labels = {}
    for house in range(1, 6):
        file_path = f'data/ukdale/house_{house}/labels.dat'
        labels[house] = {}
        with open(file_path) as f:
            for line in f:
                split_line = line.split(' ')
                channel = int(split_line[0])
                device_name = split_line[1].strip()
                prefix = 'ft_'

                if device_name == 'aggregate':
                    prefix = ''
                    device_name = 'aggregate_apparent'

                labels[house][channel] = prefix + device_name
        labels[house]['mains'] = 'mains'
    return labels
file_labels = read_labels()
# for i in range(1,3):
#     print('House {}: '.format(i), labels[i], '\n')
for house in range(1,2):
    print(f'House {house}: file_labels[i]')

House 1: file_labels[i]


In [95]:
def get_house_path(house):
    return f'data/ukdale/house_{house}/'
    
def get_chan_path(house, channel):
    if channel == 'mains':
        return get_house_path(house) + f'mains.dat'

    return get_house_path(house) + f'channel_{channel}.dat'

def get_num_apps(house):
    return len(glob.glob(get_house_path(house) + 'channel_*[0-9].dat'))

In [96]:
def read_file(house, channel):
    print(f'reading house {house}; channel {channel}');
    file = get_chan_path(house, channel)
    
    df = pd.read_table(file, sep = ' ', names = ['unix_time', file_labels[house][channel]], 
                                       dtype = {'unix_time': 'int64', file_labels[house][channel]:'float64'}) 

    return df

def read_mains_file(house):
    
    file = get_chan_path(house, 'mains')
    
    df = pd.read_table(file, sep = ' ', names = ['unix_time', 'mains_active', 'mains_apparent', 'mains_rms' ], 
                                       dtype = {'unix_time': 'float64', 'mains_active':'float64',
                                                'mains_apparent': 'float64', 'mains_rms': 'float64'}) 

    return df

In [97]:
def parse_data(df, sort_index = True, drop_duplicates = True):
    df['timestamp'] = df['unix_time'].astype("datetime64[s]")
    df.set_index(df['timestamp'].values, inplace=True)
    df.drop(['unix_time'], axis=1, inplace=True)

#     tz = 'US/Eastern'
#     # Convert the integer index column to timezone-aware datetime 
#     df.index = pd.to_datetime(df.index.values, unit='s', utc=True)
#     df = df.tz_convert(tz)

    if sort_index:
        df = df.sort_index() # if using REDD: raw data might not be sorted
        
    if drop_duplicates:
        dups_in_index = df.index.duplicated(keep='first')
        if dups_in_index.any():
#             print('Data has duplicates', house, channel, dups_in_index[dups_in_index == True])
            df = df[~dups_in_index]

    return df

In [98]:
def get_timeframe(df):
    start = df.index[0]
    end = df.index[-1]
    
    return start, end

In [99]:
def get_feature_columns(df):
    return list(filter(lambda x: x.startswith('ft_'), df.columns.tolist()))

In [100]:
def sub_sample_mains(df):
    print('Before sub sampling')
    print_heads_tails(df)
    df = df.resample('6S').max()
    print('After sub sampling')
    print_heads_tails(df)
    
    return df

In [101]:
def fill_na(df, dry_run = False):
    for label in df.columns:
        null_count = df[label].isnull().sum()
        zero_count = df[label].isin([0]).sum()
        print(f'[fill_na] checked NaN count for {label}; result is {null_count}')
        print(f'[fill_na] checked zero count for {label}; result is {zero_count}')
        if not dry_run and null_count > 0:
            df[label].interpolate(method='linear', inplace=True)
            df[label].fillna(0.0, inplace=True)
            print(f'[fill_na] post filling - NaN count for {label}; result is {null_count}')
            print(f'[fill_na] post filling - checked zero count for {label}; result is {zero_count}')

In [102]:
DATA_SIZE_LIMIT = 17000000

In [103]:
def read_mains(house):
    df = read_mains_file(house)
    df = parse_data(df)
    
    return df

In [104]:
mains_df = {}
for house in range(1,2):
    mains_df[house] = read_mains(house)
    fill_na(mains_df[house], dry_run=True)

[fill_na] checked NaN count for mains_active; result is 0
[fill_na] checked zero count for mains_active; result is 18
[fill_na] checked NaN count for mains_apparent; result is 0
[fill_na] checked zero count for mains_apparent; result is 0
[fill_na] checked NaN count for mains_rms; result is 0
[fill_na] checked zero count for mains_rms; result is 0
[fill_na] checked NaN count for timestamp; result is 0
[fill_na] checked zero count for timestamp; result is 0


In [105]:
for house in range(1,2):
    mains_df[house] = sub_sample_mains(mains_df[house])
    fill_na(mains_df[house], dry_run=True)

Before sub sampling
Data has shape: (126862421, 4)


,mains_active,mains_apparent,mains_rms,timestamp
2013-03-17 19:12:43,337.88,431.04,240.15,2013-03-17 19:12:43
2013-03-17 19:12:44,339.43,427.94,240.56,2013-03-17 19:12:44


...


,mains_active,mains_apparent,mains_rms,timestamp
2017-04-26 17:35:57,582.24,646.12,245.47,2017-04-26 17:35:57
2017-04-26 17:35:58,586.84,647.80,245.43,2017-04-26 17:35:58


After sub sampling
Data has shape: (21613433, 4)


,mains_active,mains_apparent,mains_rms,timestamp
2013-03-17 19:12:42,340.88,431.04,241.07,2013-03-17 19:12:47
2013-03-17 19:12:48,341.33,429.67,241.31,2013-03-17 19:12:53


...


,mains_active,mains_apparent,mains_rms,timestamp
2017-04-26 17:35:48,608.71,666.33,245.64,2017-04-26 17:35:53
2017-04-26 17:35:54,592.42,649.91,245.54,2017-04-26 17:35:58


[fill_na] checked NaN count for mains_active; result is 252999
[fill_na] checked zero count for mains_active; result is 0
[fill_na] checked NaN count for mains_apparent; result is 252999
[fill_na] checked zero count for mains_apparent; result is 0
[fill_na] checked NaN count for mains_rms; result is 252999
[fill_na] checked zero count for mains_rms; result is 0
[fill_na] checked NaN count for timestamp; result is 252999
[fill_na] checked zero count for timestamp; result is 0


### Interpolate missing values and re-add timestamp

pandas interpolate drops non numerical columns, so we need to re-add the timestamp column

In [106]:
for house in range(1,2):
    fill_na(mains_df[house])
    mains_df[house]['timestamp'] = mains_df[house].index
    print('')
    fill_na(mains_df[house], dry_run=True)

[fill_na] checked NaN count for mains_active; result is 252999
[fill_na] checked zero count for mains_active; result is 0
[fill_na] post filling - NaN count for mains_active; result is 252999
[fill_na] post filling - checked zero count for mains_active; result is 0
[fill_na] checked NaN count for mains_apparent; result is 252999
[fill_na] checked zero count for mains_apparent; result is 0
[fill_na] post filling - NaN count for mains_apparent; result is 252999
[fill_na] post filling - checked zero count for mains_apparent; result is 0
[fill_na] checked NaN count for mains_rms; result is 252999
[fill_na] checked zero count for mains_rms; result is 0
[fill_na] post filling - NaN count for mains_rms; result is 252999
[fill_na] post filling - checked zero count for mains_rms; result is 0
[fill_na] checked NaN count for timestamp; result is 252999
[fill_na] checked zero count for timestamp; result is 0
[fill_na] post filling - NaN count for timestamp; result is 252999
[fill_na] post filling 

In [107]:
def read_merge_data(house):
#     df = read_file(house, 1)
#     df = parse_data(df)

    df = mains_df[house]
    print(f'read house {house}; mains; df.shape is {df.shape}')

    num_apps = get_num_apps(house)
    for i in range(1, num_apps + 1):
        data = read_file(house, i)
        if data.shape[0] >= DATA_SIZE_LIMIT:
            print(f'read house {house}; channel {i}; df.shape is {df.shape}; data.shape is {data.shape}')
            data = parse_data(data)

            start_x, end_x = get_timeframe(data)
            start_y, end_y = get_timeframe(df)

            start = start_x if start_x > start_y else start_y
            end = end_x if end_x < end_y else end_y

            data = data[(data.index >= start) &
                                   (data.index <= end)]
            df = df[(df.index >= start) &
                                   (df.index <= end)]

            df = pd.merge_asof(df, data, on = 'timestamp', tolerance=pd.Timedelta('6s'))
            df.set_index(df['timestamp'].values, inplace=True)
        else:
            print(f'skipping house {house}; channel {i}; df.shape is {df.shape}; data.shape is {data.shape}')
#         df = pd.merge(df, data, how = 'inner', on = 'unix_time')

    return df

df = {}
for i in range(1,2):
    df[i] = read_merge_data(i)

read house 1; mains; df.shape is (21613433, 4)
reading house 1; channel 1
read house 1; channel 1; df.shape is (21613433, 4); data.shape is (21837636, 2)
reading house 1; channel 2
read house 1; channel 2; df.shape is (21613400, 5); data.shape is (21281331, 2)
reading house 1; channel 3
read house 1; channel 3; df.shape is (21613400, 6); data.shape is (21281208, 2)
reading house 1; channel 4
skipping house 1; channel 4; df.shape is (21613400, 7); data.shape is (4539118, 2)
reading house 1; channel 5
read house 1; channel 5; df.shape is (21613400, 7); data.shape is (19555935, 2)
reading house 1; channel 6
read house 1; channel 6; df.shape is (21613400, 8); data.shape is (19819392, 2)
reading house 1; channel 7
read house 1; channel 7; df.shape is (21613400, 9); data.shape is (19763329, 2)
reading house 1; channel 8
read house 1; channel 8; df.shape is (21613400, 10); data.shape is (21449386, 2)
reading house 1; channel 9
read house 1; channel 9; df.shape is (21613400, 11); data.shape is

In [108]:
for house in range(1,2):
    fill_na(df[house])

[fill_na] checked NaN count for mains_active; result is 0
[fill_na] checked zero count for mains_active; result is 0
[fill_na] checked NaN count for mains_apparent; result is 0
[fill_na] checked zero count for mains_apparent; result is 0
[fill_na] checked NaN count for mains_rms; result is 0
[fill_na] checked zero count for mains_rms; result is 0
[fill_na] checked NaN count for timestamp; result is 0
[fill_na] checked zero count for timestamp; result is 0
[fill_na] checked NaN count for aggregate_apparent; result is 437956
[fill_na] checked zero count for aggregate_apparent; result is 0
[fill_na] post filling - NaN count for aggregate_apparent; result is 437956
[fill_na] post filling - checked zero count for aggregate_apparent; result is 0
[fill_na] checked NaN count for ft_boiler; result is 1106866
[fill_na] checked zero count for ft_boiler; result is 2387619
[fill_na] post filling - NaN count for ft_boiler; result is 1106866
[fill_na] post filling - checked zero count for ft_boiler; 

[fill_na] post filling - NaN count for ft_data_logger_pc; result is 1314286
[fill_na] post filling - checked zero count for ft_data_logger_pc; result is 463
[fill_na] checked NaN count for ft_childs_table_lamp; result is 1361798
[fill_na] checked zero count for ft_childs_table_lamp; result is 19314642
[fill_na] post filling - NaN count for ft_childs_table_lamp; result is 1361798
[fill_na] post filling - checked zero count for ft_childs_table_lamp; result is 19314642
[fill_na] checked NaN count for ft_childs_ds_lamp; result is 2014564
[fill_na] checked zero count for ft_childs_ds_lamp; result is 17394445
[fill_na] post filling - NaN count for ft_childs_ds_lamp; result is 2014564
[fill_na] post filling - checked zero count for ft_childs_ds_lamp; result is 17394445
[fill_na] checked NaN count for ft_baby_monitor_tx; result is 2276706
[fill_na] checked zero count for ft_baby_monitor_tx; result is 18082403
[fill_na] post filling - NaN count for ft_baby_monitor_tx; result is 2276706
[fill_na

In [36]:
for house in range(1,2):
    print_heads_tails(df[house])

Data has shape: (20702300, 36)


,mains_active,mains_apparent,mains_rms,timestamp,aggregate_apparent,ft_boiler,ft_solar_thermal_pump,ft_washing_machine,ft_dishwasher,ft_tv,...,ft_bedroom_d_lamp,ft_coffee_machine,ft_kitchen_radio,ft_gas_oven,ft_data_logger_pc,ft_childs_table_lamp,ft_childs_ds_lamp,ft_baby_monitor_tx,ft_office_lamp1,ft_office_lamp2
2013-04-11 19:09:00,630.29,683.14,242.54,2013-04-11 19:09:00,675.0,0.0,0.0,0.0,1.0,97.0,...,0.0,1.0,1.0,46.0,12.0,0.0,0.0,0.0,14.0,0.0
2013-04-11 19:09:06,628.35,681.59,242.76,2013-04-11 19:09:06,679.0,12.0,0.0,0.0,1.0,96.0,...,0.0,1.0,1.0,46.0,13.0,0.0,0.0,0.0,14.0,10.0


...


,mains_active,mains_apparent,mains_rms,timestamp,aggregate_apparent,ft_boiler,ft_solar_thermal_pump,ft_washing_machine,ft_dishwasher,ft_tv,...,ft_bedroom_d_lamp,ft_coffee_machine,ft_kitchen_radio,ft_gas_oven,ft_data_logger_pc,ft_childs_table_lamp,ft_childs_ds_lamp,ft_baby_monitor_tx,ft_office_lamp1,ft_office_lamp2
2017-03-19 10:58:48,2609.15,2628.64,243.93,2017-03-19 10:58:48,2561.0,12.0,51.0,2240.0,1.0,0.0,...,0.0,1.0,1.0,3.0,13.0,0.0,8.0,0.0,0.0,0.0
2017-03-19 10:58:54,2532.95,2550.77,241.81,2017-03-19 10:58:54,2534.0,13.0,51.0,2157.0,1.0,0.0,...,0.0,1.0,1.0,3.0,13.0,0.0,8.0,0.0,1.0,0.0


### Convert apparent to real

In [44]:
power_factors = {}
apparent_labels = {
    1: ['aggregate_apparent', 'ft_boiler', 'ft_solar_thermal_pump', 'ft_kitchen_lights', 'ft_lighting_circuit']
}
for house in range(1,2):
    mains_active_sample = df[house]['mains_active'].iloc[0]
    mains_apparent_sample = df[house]['mains_apparent'].iloc[0]
    power_factors[house] = mains_active_sample / mains_apparent_sample
    for apparent_label in apparent_labels[house]:
        if apparent_label in df[house].columns:
            print(f'Converting apparent to real for {apparent_label}')
            if apparent_label == 'aggregate_apparent':
                df[house]['aggregate_active'] = df[house][apparent_label] * power_factors[house]
            else:
                df[house][apparent_label] = df[house][apparent_label] * power_factors[house]
        else:
            print(f'Cannot convert apparent to real for {apparent_label}, since it is not present in df')

Converting apparent to real for aggregate_apparent
Converting apparent to real for ft_boiler
Converting apparent to real for ft_solar_thermal_pump
Converting apparent to real for ft_kitchen_lights
Converting apparent to real for ft_lighting_circuit


In [46]:
for house in range(1,2):
    print_heads_tails(df[house])

Data has shape: (20702300, 37)


,mains_active,mains_apparent,mains_rms,timestamp,aggregate_apparent,ft_boiler,ft_solar_thermal_pump,ft_washing_machine,ft_dishwasher,ft_tv,...,ft_coffee_machine,ft_kitchen_radio,ft_gas_oven,ft_data_logger_pc,ft_childs_table_lamp,ft_childs_ds_lamp,ft_baby_monitor_tx,ft_office_lamp1,ft_office_lamp2,aggregate_active
2013-04-11 19:09:00,630.29,683.14,242.54,2013-04-11 19:09:00,675.0,0.00000,0.0,0.0,1.0,97.0,...,1.0,1.0,46.0,12.0,0.0,0.0,0.0,14.0,0.0,622.779738
2013-04-11 19:09:06,628.35,681.59,242.76,2013-04-11 19:09:06,679.0,11.07164,0.0,0.0,1.0,96.0,...,1.0,1.0,46.0,13.0,0.0,0.0,0.0,14.0,10.0,626.470284


...


,mains_active,mains_apparent,mains_rms,timestamp,aggregate_apparent,ft_boiler,ft_solar_thermal_pump,ft_washing_machine,ft_dishwasher,ft_tv,...,ft_coffee_machine,ft_kitchen_radio,ft_gas_oven,ft_data_logger_pc,ft_childs_table_lamp,ft_childs_ds_lamp,ft_baby_monitor_tx,ft_office_lamp1,ft_office_lamp2,aggregate_active
2017-03-19 10:58:48,2609.15,2628.64,243.93,2017-03-19 10:58:48,2561.0,11.071640,47.054469,2240.0,1.0,0.0,...,1.0,1.0,3.0,13.0,0.0,8.0,0.0,0.0,0.0,2362.872457
2017-03-19 10:58:54,2532.95,2550.77,241.81,2017-03-19 10:58:54,2534.0,11.994276,47.054469,2157.0,1.0,0.0,...,1.0,1.0,3.0,13.0,0.0,8.0,0.0,1.0,0.0,2337.961267


### Drop Low Power Devices

In [47]:
averages = {}
def get_devices_mean_power(df):
    for house in range(1,2):
        averages[house] = {}
        num_apps = get_num_apps(i)
        for label in get_feature_columns(df[house]):
            mean = df[house][label].mean()
            averages[house][label] = mean

get_devices_mean_power(df)

In [48]:
for house in range(1,2):
    sorted_averages = sorted(averages[house].items(), key=lambda x: x[1])
    for chan, avg in reversed(sorted_averages):
        print(chan, (25 - len(chan)) * " ",  avg)

ft_fridge                  40.95655226713941
ft_washing_machine         29.904752515421
ft_lighting_circuit        27.11549216905398
ft_dishwasher              26.023200586408272
ft_kitchen_lights          19.456376381821364
ft_htpc                    18.9795104891727
ft_boiler                  16.707694387903086
ft_kettle                  16.06120438308787
ft_data_logger_pc          13.476598493887153
ft_tv                      12.894252112084164
ft_solar_thermal_pump      9.200153255825809
ft_toaster                 8.548825855098226
ft_microwave               8.240620486612599
ft_adsl_router             7.199337513223169
ft_gas_oven                3.8930611091521232
ft_amp_livingroom          3.5832091844867477
ft_livingroom_s_lamp       3.3853455413166653
ft_subwoofer_livingroom    3.305320640701758
ft_kitchen_phone&stereo    2.8723354409896484
ft_livingroom_lamp_tv      1.4113156026142022
ft_coffee_machine          1.3653220415122957
ft_office_lamp2            1.1515563246595788
f

In [54]:
def drop_low_power_devices(df, labels, averages):
    print('drop_low_power_devices :: initial shape', df.shape)
    for label in labels:
        mean = averages[label]

        if mean < 1.0:
            print(f'dropping low power feature - {label} with mean - {mean} for house - {house} at chan - {chan}')
            df.drop([label], axis=1, inplace=True)

    print('drop_low_power_devices :: final shape', df.shape)

### Re-add unix time

It's easier to parse the unix_time than datetime from csv, so we want to re-add this before dumping to csv

In [69]:
def re_add_unix_time_feature(df, inplace=True):
    print('re-adding unix timestamp')    
    df['unix_time'] = df.timestamp.map(pd.Timestamp.timestamp)
    df.set_index(df['unix_time'].values, inplace=True)
    df.drop(['timestamp'], axis=1, inplace=True)

### Dump Data into new CSVs

In [55]:
import pathlib

newChunkDataDir = 'data/ukdale-parsed-chunks';
pathlib.Path(newChunkDataDir).mkdir(parents=True, exist_ok=True) 

for house in range(1,2):
    houseDir = f"{newChunkDataDir}/house_{house}"
    pathlib.Path(houseDir).mkdir(parents=True, exist_ok=True) 

In [61]:
def df_chunk_to_csv(house, chunk, df):
    file_name = f"{newChunkDataDir}/house_{house}/chunk_{chunk}.dat"
    df.to_csv(file_name, sep='\t', header=True, index=False)

In [66]:
import math

def save_by_chunks(df):
    file_row_size = 4500000
    for house in range(1,2):
        df_row_size = df[house].shape[0]
        no_of_chunks = math.ceil(df_row_size / file_row_size)
        print(f'Total row size - {df_row_size}; Number of chunks - {no_of_chunks}')
        drop_low_power_devices(df[house], get_feature_columns(df[house]), averages[house])
        re_add_unix_time_feature(df[house], inplace=False)
        for n in range(no_of_chunks):
            start_index = n * file_row_size
            end_index = start_index + file_row_size
            end_index = end_index if end_index <= df_row_size else df_row_size

            split_df = df[house].iloc[start_index:end_index]
            print(f'saving chunk - {n+1}, with row range - {start_index} to {end_index} and shape - {split_df.shape}')
            
            df_chunk_to_csv(house, n+1, split_df)

In [67]:
save_by_chunks(df)

Total row size - 20702300; Number of chunks - 5
drop_low_power_devices :: initial shape (20702300, 28)
drop_low_power_devices :: final shape (20702300, 28)
saving chunk - 1, with row range - 0 to 4500000 and shape - (4500000, 28)
saving chunk - 2, with row range - 4500000 to 9000000 and shape - (4500000, 28)
saving chunk - 3, with row range - 9000000 to 13500000 and shape - (4500000, 28)
saving chunk - 4, with row range - 13500000 to 18000000 and shape - (4500000, 28)
saving chunk - 5, with row range - 18000000 to 20702300 and shape - (2702300, 28)
